# Análisis de la Desigualdad Salarial: Descomposición y RIF

Este cuaderno replica el análisis del fichero `estructura productiva.do`, centrandonos en la descomposición de la varianza salarial utilizando Regresiones de Función de Influencia Recentrada (RIF) y la descomposición de Oaxaca-Blinder.

### 1. Carga de Datos e Importaciones

Cargamos los datos procesados del cuaderno anterior y las librerías necesarias para el análisis.

In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from rifreg import RIFRegression
import matplotlib.pyplot as plt

c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### 2. Preparación de Datos

Replicamos los pasos de preparación de datos del script de Stata, creando las variables para el análisis sectorial.

In [2]:
# Cargar los datos procesados
processed_data_path = os.path.join('..', 'data', 'EES_procesado.parquet')
df = pd.read_parquet(processed_data_path)

# Crear la columna lsalmes antes de usarla
df['lsalmes'] = np.log(df['salmes'])

# Creación de la variable de sector CEDEFOP (sec_ced)
secc_to_ced_map = {
    'N0': 1, 'O0': 1,
    'D0': 3,
    'E0': 4,
    'B0': 5, 'C1': 5, 'C2': 5, 'C3': 5, 'C4': 5, 'C5': 5, 'C6': 5, 'C7': 5, 'C8': 5,
    'F0': 6,
    'G1': 7, 'G2': 7,
    'I0': 8,
    'H1': 9, 'H2': 9,
    'J0': 10,
    'K0': 11, 'L0': 11,
    'M0': 12,
    'P0': 13, 'Q0': 13,
    'R0': 14,
    'S0': 15
}
df['sec_ced'] = df['secc'].map(secc_to_ced_map)

# Creación de la variable de actividad agregada (act)
df['act'] = np.nan
df.loc[df['sec_ced'] == 5, 'act'] = 'I' # Industria
df.loc[df['sec_ced'].isin([3, 4]), 'act'] = 'E' # Energía
df.loc[df['sec_ced'] == 6, 'act'] = 'C' # Construcción
df.loc[df['sec_ced'].isin([10, 11]), 'act'] = 'ST' # Servicios Tecnológicos
df.loc[df['act'].isna() & (df['sec_ced'] != 2), 'act'] = 'S' # El resto de los servicios

print("Variables 'sec_ced' y 'act' creadas.")

Variables 'sec_ced' y 'act' creadas.
